# Mathematical Underpinnings - Lab 6

In [54]:
from sklearn.metrics import mutual_info_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
#from tqdm import tqdm
from scipy import stats

## Useful functions

In [55]:
def discretize_2bins(X):
    X_discrete = 1 * (X >= 0)
    return X_discrete

In [56]:
def conditional_permutation(X, Z):
    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    X_b = np.zeros(n)

    for i in range(n_z_values):
        z_value_tmp = z_values[i]

        X_b[Z == z_value_tmp] = np.random.permutation(X[Z == z_value_tmp])

    return X_b

In [57]:
def conditional_mutual_information(X, Y, Z):
    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    cmi = 0

    for i in range(n_z_values):
        z_value_tmp = z_values[i]
        z_condition = (Z == z_value_tmp)

        X_z = X[z_condition]
        Y_z = Y[z_condition]

        mi_XY_z = mutual_info_score(X_z, Y_z)
        p_z = np.sum(z_condition) / n

        cmi += p_z * mi_XY_z

    return cmi

In [58]:
# II(X;Y;Z)
def interaction_information(X, Y, Z):
    return conditional_mutual_information(X, Y, Z) - mutual_info_score(X, Y)

In [59]:
# II(X;Y;Z1;Z2)
def interaction_information2(X, Y, Z1, Z2):
    Z_1_and_2 = 2 * Z2 + Z1
    return interaction_information(X, Y, Z_1_and_2) - interaction_information(X, Y, Z1) - interaction_information(X, Y,
                                                                                                                  Z2)

## Task 1

In [60]:
def discretize_2bins_1_minus1(X):
    X_discrete = discretize_2bins(X) * 2 - 1
    return X_discrete

In [61]:
def secmi2(X, Y, Z):
    s = 0
    for i in range(len(Z[0])):
        s += interaction_information(Y, X, Z[:, i])
    return mutual_info_score(X, Y) + s


def secmi3(X, Y, Z):
    s = 0
    for i in range(len(Z[0]) - 1):
        for j in range(i + 1, len(Z[0])):
            s += interaction_information2(X, Y, Z[:, i], Z[:, j])

    return secmi2(X, Y, Z) + s


### a)

In [62]:
def cond_indep_test_permutation(X, Y, Z, B=50, stat='cmi'):
    n_col_Z = Z.shape[1]
    Z_1dim = np.dot(Z, 2 ** np.linspace(0, n_col_Z - 1, n_col_Z))

    if stat == "cmi":
        stat_value = conditional_mutual_information(X, Y, Z_1dim)
    if stat == "secmi2":
        stat_value = secmi2(X, Y, Z)
    if stat == "secmi3":
        stat_value = secmi3(X, Y, Z)

    condition_p_value = 0
    for b in range(B):
        X_b = conditional_permutation(X, Z_1dim)

        if stat == "cmi":
            stat_value_b = conditional_mutual_information(X_b, Y, Z_1dim)
        if stat == "secmi2":
            stat_value_b = secmi2(X_b, Y, Z)
        if stat == "secmi3":
            stat_value_b = secmi3(X_b, Y, Z)

        if stat_value <= stat_value_b:
            condition_p_value += 1

    p_value = (1 + condition_p_value) / (1 + B)

    return 2 * len(X) * stat_value, p_value

### b)

In [63]:
def generate_data(seed=123):
    np.random.seed(seed=seed)

    Y = stats.norm.rvs(0, 1, size=100)
    Y_tilde = discretize_2bins_1_minus1(Y)

    Z1 = stats.norm.rvs(0, 1, size=100) + Y_tilde
    Z2 = stats.norm.rvs(0, 1, size=100) + Y_tilde
    Z3 = stats.norm.rvs(0, 1, size=100) + Y_tilde

    Z1_tilde = discretize_2bins_1_minus1(Z1)
    Z2_tilde = discretize_2bins_1_minus1(Z2)
    Z3_tilde = discretize_2bins_1_minus1(Z3)

    Z_tilde = np.hstack([Z1_tilde.reshape(-1, 1), Z2_tilde.reshape(-1, 1), Z3_tilde.reshape(-1, 1)])

    X = stats.norm.rvs(0, 1, size=100) + Z1_tilde
    X_tilde = discretize_2bins_1_minus1(X)

    return X_tilde, Y_tilde, Z_tilde

In [74]:
N = 100

df_pvalues = pd.DataFrame(
    columns=['cmi_case1', 'secmi2_case1', 'secmi3_case1', 'cmi_case2', 'secmi2_case2', 'secmi3_case2'])

for i in range(N):
    X_tilde, Y_tilde, Z_tilde = generate_data(i)

    cmi_case1 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [0, 1]], B=50, stat='cmi')
    secmi2_case1 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [0, 1]], B=50, stat='secmi2')
    secmi3_case1 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [0, 1]], B=50, stat='secmi3')

    cmi_case2 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [1, 2]], B=50, stat='cmi')
    secmi2_case2 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [1, 2]], B=50, stat='secmi2')
    secmi3_case2 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [1, 2]], B=50, stat='secmi3')

    df_pvalues.loc[i] = [cmi_case1[1], secmi2_case1[1], secmi3_case1[1], cmi_case2[1], secmi2_case2[1], secmi3_case2[1]]

df_pvalues

,cmi_case1,secmi2_case1,secmi3_case1,cmi_case2,secmi2_case2,secmi3_case2
0,0.117647,0.235294,0.156863,0.019608,0.019608,0.019608
1,0.882353,0.921569,0.941176,0.078431,0.098039,0.156863
2,0.039216,0.019608,0.039216,0.019608,0.019608,0.019608
3,0.078431,0.411765,0.039216,0.019608,0.019608,0.019608
4,0.137255,0.117647,0.235294,0.019608,0.019608,0.019608
...,...,...,...,...,...,...
95,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608
96,0.372549,0.313725,0.490196,0.215686,0.156863,0.156863
97,0.294118,0.176471,0.274510,0.137255,0.254902,0.156863
98,0.921569,0.784314,0.901961,0.019608,0.019608,0.019608


In [75]:
for test_name in ['cmi_case1', 'secmi2_case1', 'secmi3_case1', 'cmi_case2', 'secmi2_case2', 'secmi3_case2']:
    fraction_of_rejected = len(df_pvalues[test_name].loc[lambda x: x < 0.05]) / len(df_pvalues)

    print(f'{test_name} rejected in {fraction_of_rejected} cases')

cmi_case1 rejected in 0.04 cases
secmi2_case1 rejected in 0.04 cases
secmi3_case1 rejected in 0.03 cases
cmi_case2 rejected in 0.6 cases
secmi2_case2 rejected in 0.58 cases
secmi3_case2 rejected in 0.6 cases


Only the first conditional independence is true. We can observe that in test results. However, even in the second case hypothesis was rejected only around 60% of times.

### c)

In [92]:
def generate_data2(seed=123):
    np.random.seed(seed=seed)

    X = stats.binom.rvs(1, 0.5, size=100)
    Z1 = stats.binom.rvs(1, 0.5, size=100)
    Z2 = stats.binom.rvs(1, 0.5, size=100)
    Z3 = stats.binom.rvs(1, 0.5, size=100)

    Z = np.hstack([Z1.reshape(-1, 1), Z2.reshape(-1, 1), Z3.reshape(-1, 1)])

    Y = stats.binom.rvs(1, np.where(np.mod(X + Z1 + Z2, 2), 0.8, 0.2))

    return X, Y, Z

In [96]:
N = 100

df_pvalues = pd.DataFrame(
    columns=['cmi_case1', 'secmi2_case1', 'secmi3_case1', 'cmi_case2', 'secmi2_case2', 'secmi3_case2'])

for i in range(N):
    X_tilde, Y_tilde, Z_tilde = generate_data2(i)

    cmi_case1 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [0, 1]], B=50, stat='cmi')
    secmi2_case1 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [0, 1]], B=50, stat='secmi2')
    secmi3_case1 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [0, 1]], B=50, stat='secmi3')

    cmi_case2 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [1, 2]], B=50, stat='cmi')
    secmi2_case2 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [1, 2]], B=50, stat='secmi2')
    secmi3_case2 = cond_indep_test_permutation(X_tilde, Y_tilde, Z_tilde[:, [1, 2]], B=50, stat='secmi3')

    df_pvalues.loc[i] = [cmi_case1[1], secmi2_case1[1], secmi3_case1[1], cmi_case2[1], secmi2_case2[1], secmi3_case2[1]]

df_pvalues

,cmi_case1,secmi2_case1,secmi3_case1,cmi_case2,secmi2_case2,secmi3_case2
0,0.019608,0.588235,0.019608,0.313725,0.352941,0.196078
1,0.019608,0.039216,0.019608,0.156863,0.039216,0.117647
2,0.019608,0.921569,0.019608,0.862745,0.705882,0.862745
3,0.019608,0.784314,0.019608,0.980392,0.882353,0.823529
4,0.019608,0.450980,0.019608,0.549020,0.294118,0.392157
...,...,...,...,...,...,...
95,0.019608,0.470588,0.019608,1.000000,0.921569,0.980392
96,0.019608,0.588235,0.019608,0.921569,0.882353,0.921569
97,0.019608,0.666667,0.019608,0.372549,0.745098,0.274510
98,0.019608,1.000000,0.019608,0.568627,0.450980,0.647059


In [97]:
for test_name in ['cmi_case1', 'secmi2_case1', 'secmi3_case1', 'cmi_case2', 'secmi2_case2', 'secmi3_case2']:
    fraction_of_rejected = len(df_pvalues[test_name].loc[lambda x: x < 0.05]) / len(df_pvalues)

    print(f'{test_name} rejected in {fraction_of_rejected} cases')

cmi_case1 rejected in 1.0 cases
secmi2_case1 rejected in 0.04 cases
secmi3_case1 rejected in 1.0 cases
cmi_case2 rejected in 0.05 cases
secmi2_case2 rejected in 0.03 cases
secmi3_case2 rejected in 0.02 cases


Conditional independence is true in the second case. The SECMI2 test however is unable to detect 3-way interactions, therefore, as in this case this is exactly what we deal with, it is unable to perform correctly and reject the hypothesis in the first case.

## Task 2
 
in R